In [1]:
import pandas as pd
import json



df = pd.read_csv("JoSAA 2022 Seat Allocation Tool - For Surya.csv", header=0)
# print(df.head())
seat_types = df["Seat Type"].unique()
print(seat_types)

for seat_type in seat_types:
    df_seat_type = df[df["Seat Type"] == seat_type]
    df_seat_type.to_json(f"public/data/{seat_type}.json", orient="records")

['OPEN' 'EWS' 'OBC-NCL' 'SC' 'ST' 'OPEN (PwD)' 'EWS (PwD)' 'OBC-NCL (PwD)'
 'SC (PwD)' 'ST (PwD)']


In [42]:
df.columns

Index(['College Rank', 'State', 'Exam', 'Institute', 'Academic Program Name',
       'Quota', 'Special Quota', 'Seat Type', 'Gender', 'Round',
       'Opening Rank', 'Closing Rank', 'Input Rank Comparison to Closing Rank',
       'Category Match', '500', 'OBC-NCL'],
      dtype='object')

In [18]:
df["Gender"].unique()

array(['Gender-Neutral', 'Female-only (including Supernumerary)'],
      dtype=object)

In [22]:
df["Exam"].unique()

array(['JEE Advanced', 'JEE Main'], dtype=object)

In [23]:
df["State"].unique()

array(['Odisha', 'Maharashtra', 'Himachal Pradesh', 'New Delhi',
       'Madhya Pradesh', 'West Bengal', 'Telangana', 'Rajasthan',
       'Uttar Pradesh', 'Tamil Nadu', 'Gujarat', 'Bihar', 'Uttarakhand',
       'Jharkhand', 'Punjab', 'Assam', 'Chhatisgarh', 'Goa', 'Kerala',
       'Andhra Pradesh', 'Jammu & Kashmir', 'Karnataka', 'Tripura',
       'Meghalaya', 'Nagaland', 'Puducherry', 'Sikkim',
       'Arunachal Pradesh', 'Haryana', 'Manipur', 'Mizoram', 'Diu'],
      dtype=object)

In [19]:
df["Round"].max()

6

In [28]:
df["Quota"].unique()

array(['AI', 'HS', 'OS'], dtype=object)

In [ ]:
# re rank based on round 1 ke open category ka closing ranks!

In [27]:
df[df["Institute"] == "National Institute of Technology Karnataka, Surathkal"]["Quota"].unique()

array(['HS', 'OS'], dtype=object)

In [32]:
neet_report = pd.read_csv("Admitted Candidates List Upto Mop Up Round MBBS BD.xlsx - Report.csv")
neet_stud = pd.read_csv("Admitted Candidates List Upto Mop Up Round MBBS BD.xlsx - Table 1.csv", header=1)

neet_stud.columns, neet_report.columns

(Index(['S. No.', 'Rollno', 'Name', 'Quota', 'AIR', 'Category', 'optionN o',
        'Institute Code', 'Institute Name', 'Subject', 'Allotted Category',
        'Allotte d ph', 'Admitte dRound'],
       dtype='object'),
 Index(['Institute Name', 'State', 'Subject', 'General', 'EW', 'SC', 'ST',
        'OBC', 'General_PWD', 'EW_PWD', 'SC_PWD', 'ST_PWD', 'OBC_PWD',
        'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
        'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
        'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24'],
       dtype='object'))

In [186]:
from collections import defaultdict

institute_code_to_names = defaultdict(list)
institute_name_to_codes = {}
for it, row in neet_stud.iterrows():
    code, name = row["Institute Code"], row["Institute Name"]

    if name not in institute_code_to_names[code]:
        institute_code_to_names[code].append(name)

    institute_name_to_codes[name] = code

In [ ]:
for k, v in institute_code_to_names.items():
    if len(v) > 1:
        print(k, v)
        break

In [167]:
# i am picking the first one that appears
code_to_college_mapper = {}
for k, v in institute_code_to_names.items():
    code_to_college_mapper[k] = v[0]

assert len(institute_code_to_names) == len(code_to_college_mapper)

In [168]:
# for each round, for each subject, for each quota, 
# using ph as pwd -- physically handicapped
# consider allotted cat or actual cat?
# what to do about quota? Employee State Insurance, etc.
# all of this should be for each institute na ... institute code!!
deter_cols = ["Quota", "Admitte dRound", "Allotte d ph", "Subject"]

output_cols = ["Institute Code", "Institute Name"]

close_mapper = defaultdict(lambda : 0)
open_mapper = defaultdict(lambda : float("inf"))

for it, row in neet_stud.iterrows():

    deter_col_vals = [row[x] for x in deter_cols]
    
    cat = row["Allotted Category"]
    if cat == "OP":
        cat = "General"
    
    deter_col_vals.append(cat)

    inst_code = row["Institute Code"]
    output_col_vals = [inst_code, code_to_college_mapper[inst_code]]

    all_cols = deter_col_vals + output_col_vals

    all_cols = tuple(all_cols) # hashable

    if close_mapper[all_cols] < row["AIR"]:
        close_mapper[all_cols] = row["AIR"]

    if open_mapper[all_cols] > row["AIR"]:
        open_mapper[all_cols] = row["AIR"]




In [169]:
for k, v in close_mapper.items():
    print(k, v)
    break

('Open Seat Quota', 1, 'NO', 'MBBS', 'General', 200502, 'AIIMS, New Delhi,AIIMS ANSARI NAGAR EAST AUROBINDO MARG NEW DELHI 110029, Delhi (NCT), 110029') 55.0


In [170]:
cat_unique_values = ["SC", "ST", "General", "OBC", "EW"]

In [171]:
all_dic = []
selectors = [0, 1, 2, 3, 4, 5, 6]
selector_names = ["Quota", "Round", "is_PWD", "Subject", "Seat Type", "Institute Code", "Institute Name"]

for k, v in close_mapper.items():
    dic = {}
    for key_index, name in zip(selectors, selector_names):
        dic[name] = k[key_index]

    if k in open_mapper:
        dic["Opening Rank"] = open_mapper[k]

    dic["Closing_Rank"] = v
    
    all_dic.append(dic)

    # break

all_dic = pd.DataFrame(all_dic)

In [172]:
# time to rank institutes!!
# find round 1 ka opening rank.. rank based on it!

code_to_open_ranks = defaultdict(lambda : float("inf"))

for it, row in all_dic.iterrows():
    # print(row["Round"] == 1 , row["Category"] == "General" , row["is_PWD"] == "NO", row["Quota"] == "Open Seat Quota")
    if code_to_open_ranks[row["Institute Code"]] > row["Opening Rank"]:
        code_to_open_ranks[row["Institute Code"]] = row["Opening Rank"]

        # break

In [173]:
# now sort the dict based on this
# sort keys of dict based on values
ranking = sorted(code_to_open_ranks.items(), key = lambda item: item[1])

code_to_rank_mapper = {}

rank_idx = 1
for k, v in ranking:
    code_to_rank_mapper[k] = rank_idx
    rank_idx += 1

In [174]:
len(all_dic["Institute Code"].unique()), len(code_to_open_ranks)

(483, 483)

In [175]:
all_dic["College Rank"] = all_dic["Institute Code"].map(code_to_rank_mapper)

In [177]:
# combine pwd and category
all_dic["Seat Type"] = all_dic.apply(lambda row: row["Seat Type"] + " (PwD)" if row["is_PWD"] != "NO" else row["Seat Type"], axis = 1)

In [151]:
all_dic.columns, df.columns

(Index(['Quota', 'Round', 'is_PWD', 'Subject', 'Category', 'Institute Code',
        'Institute Name', 'Opening Rank', 'Closing_Rank', 'College Rank'],
       dtype='object'),
 Index(['College Rank', 'State', 'Exam', 'Institute', 'Academic Program Name',
        'Quota', 'Special Quota', 'Seat Type', 'Gender', 'Round',
        'Opening Rank', 'Closing Rank', 'Input Rank Comparison to Closing Rank',
        'Category Match', '500', 'OBC-NCL'],
       dtype='object'))

In [183]:
all_dic = all_dic.rename(columns={
    "Category": "Seat Type",
    "Institute Name": "Institute",
    "Subject": "Academic Program Name",
    "Closing_Rank": "Closing Rank"
}) 

In [179]:
def replace_cat(row):
    cat = row["Seat Type"]
    if cat == "General":
        cat = "OPEN"
    elif cat == "General (PwD)":
        cat = "OPEN (PwD)"
    elif cat == "EW":
        cat = "EWS"
    elif cat == "EW (PwD)":
        cat = "EWS (PwD)"
    elif cat == "OBC":
        cat = "OBC-NCL"
    elif cat == "OBC (PwD)":
        cat = "OBC-NCL (PwD)"

    return cat

all_dic["Seat Type"] = all_dic.apply(lambda row: replace_cat(row), axis = 1)



In [187]:
# figuring out state for each..
# merge on institute name
print(neet_stud.shape, neet_report.shape)
inst_code_to_state_map = {}
for it, row in pd.merge(neet_stud, neet_report, on=["Institute Name"]).iterrows():
    name, state = row["Institute Name"], row["State"]

    inst_code_to_state_map[institute_name_to_codes[name]] = state



all_dic["State"] = all_dic["Institute Code"].map(inst_code_to_state_map)


(23059, 13) (433, 25)


In [188]:
for cat in all_dic["Seat Type"].unique():
    all_dic_category = all_dic[all_dic["Seat Type"] == cat]

    all_dic_category.to_json(f"public/data/NEET/{cat}.json", orient="records")

In [70]:
(1, 2, 3) + (4,)

(1, 2, 3, 4)

In [190]:
all_dic.to_csv("Surya - NEET Data.csv")

In [8]:
schol_data = pd.read_csv("scholarship_data.csv")

In [9]:
# update state names
names_in_sheet = []
for it, row in schol_data.iterrows():
    for k, v in row.items():
        if "State" in k:
            if v not in names_in_sheet:
                names_in_sheet.append(v)

            

In [10]:
names_in_sheet

[nan,
 'Andhra Pradesh',
 'Karnataka',
 'Telangana',
 'Gujarat',
 'Maharashtra',
 'Tamil Nadu',
 'Bihar',
 'Haryana',
 'Kerala',
 'Arunachal Pradesh',
 'Assam',
 'Manipur',
 'Meghalaya',
 'Mizoram',
 'Nagaland',
 'Sikkim',
 'Tripura']

In [11]:
# update state names
city_names_in_sheet = []
for it, row in schol_data.iterrows():
    for k, v in row.items():
        if "City" in k:
            if v not in city_names_in_sheet:
                city_names_in_sheet.append(v)

            

In [12]:
city_names_in_sheet

[nan, 'Mumbai', 'Delhi NCR', 'Ahmedabad', 'Bangalore', 'Pune']

In [14]:
college_names_in_sheet = []
for it, row in schol_data.iterrows():
    for k, v in row.items():
        if "College" in k:
            if v not in college_names_in_sheet:
                college_names_in_sheet.append(v)

college_names_in_sheet

['IIT', 'NIT', nan]

In [15]:
schol_data.shape

(98, 38)

,Sl No,Scholarship Name,National Scholarship,Status,Class 11 can Apply,Class 12 can Apply,Gender,Family Income (in LPA),Category,State 1,...,Benefits,Doc Required,Number of Applicants,Number of Scholars,Karnataka Applicants,Application Link,Added to Portal,Information about instituions eligble,Deadline for the year 2021-22,Unnamed: 37
0,1,Samsung Star,No,Closed,No,No,Both,NaN,NaN,NaN,...,The selected students will receive a scholarsh...,Self-attested photograph\nSelf-certified Class...,NaN,NaN,NaN,https://www.samsung.com/in/microsite/sapne-hue...,Yes,Not updated yet,NaN,NaN
1,2,Colgate - BDS,No,Closed,No,No,Both,5.0,NaN,NaN,...,"INR 50,000 per annum for 4 years",Passport-sized photograph\nValid ID proof – Ei...,NaN,NaN,NaN,https://www.colgate.com/en-in/smile-karo-aur-s...,NaN,NaN,NaN,NaN
2,3,Colgate - Class XI,No,Closed,Yes,No,Both,5.0,NaN,NaN,...,"INR 20,000 per annum for 2 years",Passport-sized photograph\nValid ID proof – Ei...,NaN,NaN,NaN,https://www.colgate.com/en-in/smile-karo-aur-s...,NaN,NaN,NaN,NaN
3,4,Colgate - Graduate,No,Closed,No,No,Both,5.0,NaN,NaN,...,"INR 30,000 per year for 3 years",Passport-sized photograph\r\nValid ID proof – ...,NaN,NaN,NaN,https://www.colgate.com/en-in/smile-karo-aur-s...,NaN,NaN,NaN,NaN
4,5,Santoor Women's,No,Closed,No,No,Female,NaN,NaN,Andhra Pradesh,...,The winners of the scholarship will be given I...,One recent passport size photograph\nPhotocopy...,NaN,NaN,NaN,https://santoorscholarships.com/index.php,NaN,Not updated yet,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,14,CENTRAL SECTOR SCHEME OF SCHOLARSHIPS FOR COLL...,Yes,NaN,No,No,Both,4.5,General,NaN,...,"12,000 per annum for all three years of UG and...","Student photograph, copy of bank passbook, Inc...",NaN,NaN,NaN,https://scholarships.gov.in/,NaN,NaN,NaN,NaN
94,15,Prime Minister's Scholarship Scheme For Centra...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://scholarships.gov.in/,NaN,NaN,NaN,NaN
95,16,PRIME MINISTER'S SCHOLARSHIP SCHEME FOR WARDS ...,Yes,NaN,No,No,Both,NaN,General,NaN,...,"2,5000 per month for male students and 3,000 p...",A certificate issued by the State Govt. indica...,NaN,NaN,NaN,https://scholarships.gov.in/,NaN,NaN,NaN,NaN
96,17,Prime Minister's Scholarship Scheme For RPF/RPSF,Yes,NaN,No,No,Both,NaN,General,NaN,...,"2,5000 per month for male students and 3,000 p...",Service Certificate issued by respective offic...,NaN,NaN,NaN,https://scholarships.gov.in/,NaN,NaN,NaN,NaN


In [19]:
schol_data.to_json("public/data/scholarships/scholarship_data.json", orient="records")